In [2]:
import numpy as np 
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy.linalg import det, svd, inv, norm
from scipy import stats
import random

## PCA + Bayes Classifier (smile vs nuetral)
def bayes_expressions(split_ratio, pca_dim):
    CLASSES=2
    PCA_DIM=pca_dim
    SPLIT_RATIO=split_ratio

    TRAIN=round(200*SPLIT_RATIO)
    TEST=200-TRAIN

    TRAIN_SAMPLES=CLASSES*TRAIN
    TEST_SAMPLES=CLASSES*TEST

    face_data = loadmat('data_mat.mat')
    face_data['face'].size

    ntrl_faces = np.zeros((200,504))
    for i in range(0,600,3):
        ntrl_faces[i//3,:] = face_data['face'][:,:,i].reshape([504])

    smile_faces = np.zeros((200,504))
    for i in range(1,600,3):
        smile_faces[i//3,:] = face_data['face'][:,:,i].reshape([504])


    data = np.zeros((200,504)) 
    data = np.concatenate((ntrl_faces,smile_faces))
    U, sigma, V = svd(data.transpose())
    Y = np.dot(data, U[:,0:PCA_DIM])

    Y_ntrl = Y[0:200]
    Y_smile = Y[200:400]

    random.shuffle(ntrl_faces);
    ntrl_faces_train = Y_ntrl[0:TRAIN]
    ntrl_faces_test = Y_ntrl[TRAIN:]

    random.shuffle(smile_faces);
    smile_faces_train = Y_smile[0:TRAIN]
    smile_faces_test = Y_smile[TRAIN:]

    test_set = np.concatenate((ntrl_faces_test, smile_faces_test), axis=0)
    test_labels = np.concatenate((np.zeros((TEST)), np.ones((TEST))))

    mean = {}
    var = {}

    mean[0]=np.mean(ntrl_faces_train,axis=0)
    var[0]=np.var(ntrl_faces_train,axis=0)

    mean[1]=np.mean(smile_faces_train,axis=0)
    var[1]=np.var(smile_faces_train,axis=0)

    def gaussian_density(X, mean, var):
        features = X.shape
        exponent = np.exp(-((X-mean) ** 2) / (2 * var))
        return (1 / (np.sqrt(2 * np.pi * var))) * exponent

    densities = np.zeros((CLASSES,TEST_SAMPLES))
    for j in range(0,TEST_SAMPLES):
        for i in range(0,CLASSES):
            densities[i,j] = np.sum(np.log(gaussian_density(test_set[j],mean[i],var[i])))

    predictions = np.argmax(densities, axis = 0)

    right=0
    wrong=0
    for i in range(0,TEST_SAMPLES):
        if predictions[i] == test_labels[i]:
            right+=1
        else:
            wrong+=1

    print("Correct : {} Incorrect : {} Accuracy : {} %".format(right, wrong, right/TEST_SAMPLES * 100))

for split_ratio in [0.95, 0.90, 0.80, 0.70, 0.60, 0.50]:
    for pca_dim in [10, 20, 30, 40 ,50 ,60]:
        bayes_expressions(split_ratio,pca_dim)

Correct : 16 Incorrect : 4 Accuracy : 80.0 %
Correct : 16 Incorrect : 4 Accuracy : 80.0 %
Correct : 17 Incorrect : 3 Accuracy : 85.0 %
Correct : 17 Incorrect : 3 Accuracy : 85.0 %
Correct : 15 Incorrect : 5 Accuracy : 75.0 %
Correct : 15 Incorrect : 5 Accuracy : 75.0 %
Correct : 15 Incorrect : 5 Accuracy : 75.0 %
Correct : 15 Incorrect : 5 Accuracy : 75.0 %
Correct : 14 Incorrect : 6 Accuracy : 70.0 %
Correct : 14 Incorrect : 6 Accuracy : 70.0 %
Correct : 15 Incorrect : 5 Accuracy : 75.0 %
Correct : 15 Incorrect : 5 Accuracy : 75.0 %
Correct : 31 Incorrect : 9 Accuracy : 77.5 %
Correct : 31 Incorrect : 9 Accuracy : 77.5 %
Correct : 35 Incorrect : 5 Accuracy : 87.5 %
Correct : 35 Incorrect : 5 Accuracy : 87.5 %
Correct : 32 Incorrect : 8 Accuracy : 80.0 %
Correct : 32 Incorrect : 8 Accuracy : 80.0 %
Correct : 31 Incorrect : 9 Accuracy : 77.5 %
Correct : 31 Incorrect : 9 Accuracy : 77.5 %
Correct : 32 Incorrect : 8 Accuracy : 80.0 %
Correct : 32 Incorrect : 8 Accuracy : 80.0 %
Correct : 